In [17]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# Step 1: 데이터 전처리 및 벡터 생성
# CSV 파일을 불러옵니다.
df = pd.read_csv('survey_data.csv')

# 원본 파일의 컬럼명인 '다음_휴가_경험'을 제외한 모든 컬럼을 특징(feature)으로 사용합니다.
# 유사도 점수를 높이기 위해 핵심 특징 5가지만 선택합니다.
selected_features = ['연령대', '성별', '함께한_사람', '휴가_장소_국내_해외', '가장_최근_여름_휴가']
features_df = df[selected_features]

# 선택된 특징 컬럼들을 원-핫 인코딩합니다.
features_encoded = pd.get_dummies(features_df)

# 모든 컬럼을 기준으로 인코딩된 전체 데이터프레임을 만듭니다.
# 이는 새로운 고객의 데이터를 인코딩할 때 기준이 됩니다.
full_encoded_df = pd.get_dummies(df)

In [8]:
# Step 2: 새로운 고객의 데이터 준비
# 새로운 고객의 가상 데이터를 만듭니다.

## 백엔드 완성되면 DB에서 설문조사한 고객의 데이터 바로 끌어올 예정.
## 현재 아래의 데이터는 임시 데이터로 저장해 둔 상태, 나중에 수정 예정.
new_user_data = {
    '연령대': '20대',
    '성별': '여성',
    '가장_최근_여름_휴가': '해수욕, 물놀이 (바다/섬 여행)',
    '휴가_장소_국내_해외': '국내',
    '휴가_장소': '제주',
    '주요_교통수단': '비행기',
    '휴가_기간': '4~6일',
    '함께한_사람': '친구',
    '총_비용': '10만~30만 원',
    '만족도': '매우 만족',
    '다음_휴가_경험': '도시 관광 (쇼핑, 카페, 시내 구경)'
}

# 새로운 고객의 데이터를 데이터프레임으로 변환합니다.
new_user_df = pd.DataFrame([new_user_data])

# 기존 데이터와 동일한 형식으로 새로운 고객 데이터를 인코딩합니다.
new_user_encoded = pd.get_dummies(new_user_df).reindex(columns=full_encoded_df.columns, fill_value=0)

# 분석에서 제외하기로 한 컬럼을 제거합니다.
# '다음_희망_휴가' 컬럼과 선택되지 않은 다른 컬럼들을 모두 제거합니다.
all_cols_to_drop = [col for col in full_encoded_df.columns if col not in features_encoded.columns]
new_user_features = new_user_encoded.drop(columns=all_cols_to_drop)

In [20]:
# Step 3: 코사인 유사도 측정
# 새로운 고객과 기존 모든 고객 간의 유사도를 계산합니다.
similarity_scores = cosine_similarity(new_user_features, features_encoded)

In [22]:
# Step 4: 가장 유사한 고객 찾기
# 유사도 점수를 내림차순으로 정렬하여 가장 높은 점수를 받은 고객의 인덱스를 찾습니다.
# [::-1]은 순서를 뒤집어 유사도가 높은 순으로 정렬합니다.
# [1:6]은 자기 자신을 제외한 상위 5명을 의미합니다.
top_indices = similarity_scores[0].argsort()[::-1][1:6]

In [23]:
# Step 5: 추천 결과 도출 및 제공
print("\n--- 가장 유사한 고객 정보 ---")
for i, index in enumerate(top_indices):
    print(f"\n[{i+1}위 고객 정보]")
    print(f"유사도 점수: {similarity_scores[0][index]:.2f}")
    
    # 해당 고객의 원본 데이터에서 정보를 가져와 출력합니다.
    similar_user_info = df.iloc[index]
    for col in similar_user_info.index:
        print(f"  - {col}: {similar_user_info[col]}")


--- 가장 유사한 고객 정보 ---

[1위 고객 정보]
유사도 점수: 0.80
  - 연령대: 20대
  - 성별: 여성
  - 가장_최근_여름_휴가: 맛집 투어 (맛집 탐방, 지역 특산물 체험)
  - 휴가_장소_국내_해외: 해외
  - 휴가_장소: 동남아시아
  - 주요_교통수단: 버스
  - 휴가_기간: 7일~ 15일
  - 함께한_사람: 혼자
  - 총_비용: 50만~100만 원
  - 만족도: 불만족
  - 다음_희망_휴가: 휴양·힐링 (스파, 리조트, 펜션 휴식)

[2위 고객 정보]
유사도 점수: 0.80
  - 연령대: 20대
  - 성별: 여성
  - 가장_최근_여름_휴가: 해수욕, 물놀이 (바다/섬 여행)
  - 휴가_장소_국내_해외: 해외
  - 휴가_장소: 동아시아
  - 주요_교통수단: 자동차
  - 휴가_기간: 2~3일
  - 함께한_사람: 친구
  - 총_비용: 100만 원~200만 원
  - 만족도: 만족
  - 다음_희망_휴가: 맛집 투어 (맛집 탐방, 지역 특산물 체험)

[3위 고객 정보]
유사도 점수: 0.80
  - 연령대: 20대
  - 성별: 여성
  - 가장_최근_여름_휴가: 해수욕, 물놀이 (바다/섬 여행)
  - 휴가_장소_국내_해외: 해외
  - 휴가_장소: 동아시아
  - 주요_교통수단: 항공편
  - 휴가_기간: 4~6일
  - 함께한_사람: 가족
  - 총_비용: 30만~50만 원
  - 만족도: 매우 만족
  - 다음_희망_휴가: 도시 관광 (쇼핑, 카페, 시내 구경)

[4위 고객 정보]
유사도 점수: 0.80
  - 연령대: 20대
  - 성별: 여성
  - 가장_최근_여름_휴가: 등산, 캠핑 등 아웃도어 활동 (산/계곡 여행)
  - 휴가_장소_국내_해외: 해외
  - 휴가_장소: 북미
  - 주요_교통수단: 자동차
  - 휴가_기간: 4~6일
  - 함께한_사람: 혼자
  - 총_비용: 30만~50만 원
  - 만족도: 만족
  - 다음_희망_휴가: 친척·지인 방문

[5위 고객 정보]
유사도